In [ ]:
# =========================================
# 1. Preparación del entorno
# =========================================
!pip install -q transformers accelerate bitsandbytes peft datasets


In [ ]:
# =========================================
# 2. Imports
# =========================================
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
#Parameter efficient fine tunning
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [ ]:
# =========================================
# 3. Modelo base (Mistral 7B Instruct)
# =========================================
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
#It’s a Hugging Face method that loads the right tokenizer for a given model.
#A tokenizer is what converts text into numbers (tokens) that the model understands, and back again
model = AutoModelForCausalLM.from_pretrained(
    model_name,#Model name
    load_in_4bit=True,#Instead of loading the model in full 16-bit or 32-bit precision (which is huge in memory), it loads weights in 4-bit precision.
    device_map="auto" #Hugging Face will automatically place the model layers across available devices (GPUs/CPU).
)

# Preparar modelo para fine-tuning LoRA
"""
When a model is quantized (load_in_4bit=True or load_in_8bit=True), some layers are stored in compressed form.

Standard LoRA training expects full-precision weights to hook into.

prepare_model_for_kbit_training prepares the model so that LoRA can safely inject its trainable adapters, without messing up the quantized weights
"""
model = prepare_model_for_kbit_training(model)

#LoRA = Low-Rank Adaptation.
#Es una técnica de fine-tuning eficiente para LLMs, que permite adaptar un modelo grande a un nuevo dominio o estilo sin entrenar todos los parámetros.
lora_config = LoraConfig(
    r=8, #Rank de la matriz de adaptación
    lora_alpha=16, #Factor de escala, ajusta la magnitud de los gradientes de LoRA
    target_modules=["q_proj","v_proj"],#Aplica LoRA a las capas de query y value projection. Esto es típico en Transformers (atención), porque estas capas son las más críticas para el “comportamiento” del modelo.
    lora_dropout=0.05, #Evitar overfitting
    bias="none",
    task_type="CAUSAL_LM" #predecir siguiente token
)
#Su objetivo es inyectar la adaptación LoRA dentro del modelo base, usando la configuración que definiste.
model = get_peft_model(model, lora_config)

In [ ]:
# =========================================
# 4. Dataset Vegeta-like (ejemplo mini)
#    👉 Reemplaza con tu propio archivo JSONL
# =========================================

import json
with open("vegeta_dataset.jsonl", "w") as f:
    for row in sample_data:
        f.write(json.dumps(row) + "\n")

dataset = load_dataset("json", data_files="vegeta_dataset.jsonl")

def format_example(example):
    return {"text": f"{example['instruction']}\nVegeta: {example['response']}"}

dataset = dataset.map(format_example)
train_data = dataset["train"]

In [ ]:
=========================================
# 5. Entrenamiento LoRA
# =========================================
training_args = TrainingArguments(
    output_dir="./vegeta-lora",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    tokenizer=tokenizer
)

trainer.train()

# Guardar LoRA en Drive
model.save_pretrained("/content/drive/MyDrive/vegeta-lora")

In [ ]:
# =========================================
# 6. Uso del modelo Vegeta
# =========================================
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True, device_map="auto")
model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/vegeta-lora")
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "Usuario: No quiero hacer más, estoy cansado.\nVegeta:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=80)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))